# Load segmentation, localise and save as h5

Segment a stack of images and then manually label a couple, then check how well the model segmented them.

In [1]:
import napari
import cellpose
from octopuslite import utils, tile
import numpy as np

import sys
sys.path.append('macrohet/')
from notify import send_sms

def view(img):
    return napari.Viewer().add_image(img)

from tqdm.auto import tqdm

import btrack
import dask.array as da

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [ ]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [ ]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
utils.read_harmony_metadata(metadata_path, assay_layout=True)

### Define row and column of choice

In [ ]:
row = '6'
column = '9'

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [ ]:
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             #set_channel=1, 
                             set_plane = 'sum_proj',
#                              set_time = 1
                         )#.astype(uint8)

#### Reorder the channel axis for localisation

In [6]:
gfp = images[:,0,...]
rfp = images[:,1,...]
images = da.stack([gfp,rfp], axis = -1)
images

,Array,Chunk
Bytes,40.88 GiB,31.01 MiB
Shape,"(75, 6048, 6048, 2)","(1, 2016, 2016, 1)"
Count,18900 Tasks,1350 Chunks
Type,uint64,numpy.ndarray


### Load masks

In [7]:
masks = np.load('segmentation/masks.npy',)

# Quick size comparison

In [13]:
import os
from skimage.io import imsave


In [12]:
os.mkdir('16bit_masks')

In [14]:
masks_16bit = np.load('segmentation/masks.npy',)
for n, frame in tqdm(enumerate(masks_16bit), total = len(masks)):
    imsave(f'16bit_masks/{n}.tiff', frame)

  0%|          | 0/75 [00:00<?, ?it/s]

/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/0.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/1.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/2.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/3.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/4.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/5.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/6.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/

/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/62.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/63.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/64.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/65.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/66.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/67.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit_masks/68.tiff is a low contrast image
  imsave(f'16bit_masks/{n}.tiff', frame)
/tmp/ipykernel_9006/1128049116.py:3: UserWarning: 16bit

# 5gb 

In [21]:
### now as semantic segmentation
from skimage.morphology import label, binary_erosion

In [19]:
!mkdir semantic_masks

In [22]:
masks_16bit = np.load('segmentation/masks.npy',)
for n, frame in tqdm(enumerate(masks_16bit), total = len(masks)):
    semantic_mask = np.zeros(frame.shape, dtype = np.uint8)
    for segment_ID in tqdm(range(1,np.max(frame)), leave = False):
        segment = frame == segment_ID
        eroded_segment = binary_erosion(segment)
        semantic_mask += eroded_segment.astype(np.uint8)
    imsave(f'semantic_masks/{n}.tiff', semantic_mask)

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

/tmp/ipykernel_9006/3873993405.py:8: UserWarning: semantic_masks/0.tiff is a low contrast image
  imsave(f'semantic_masks/{n}.tiff', semantic_mask)


  0%|          | 0/683 [00:00<?, ?it/s]

/tmp/ipykernel_9006/3873993405.py:8: UserWarning: semantic_masks/1.tiff is a low contrast image
  imsave(f'semantic_masks/{n}.tiff', semantic_mask)


...

/tmp/ipykernel_9006/3873993405.py:8: UserWarning: semantic_masks/2.tiff is a low contrast image
  imsave(f'semantic_masks/{n}.tiff', semantic_mask)


  0%|          | 0/782 [00:00<?, ?it/s]

/tmp/ipykernel_9006/3873993405.py:8: UserWarning: semantic_masks/3.tiff is a low contrast image
  imsave(f'semantic_masks/{n}.tiff', semantic_mask)


  0%|          | 0/793 [00:00<?, ?it/s]


KeyboardInterrupt



# look to be half the size, what if remove small segments?, they are 8bit

In [25]:
from skimage.morphology import remove_small_objects

In [26]:
!mkdir semantic_masks_cleaned

In [27]:
masks_16bit = np.load('segmentation/masks.npy',)
for n, frame in tqdm(enumerate(masks_16bit), total = len(masks)):
    semantic_mask = np.zeros(frame.shape, dtype = np.uint8)
    frame = remove_small_objects(frame, min_size=2500)
    for segment_ID in tqdm(range(1,np.max(frame)), leave = False):
        segment = frame == segment_ID
        eroded_segment = binary_erosion(segment)
        semantic_mask += eroded_segment.astype(np.uint8)
    imsave(f'semantic_masks_cleaned/{n}.tiff', semantic_mask)

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/769 [00:00<?, ?it/s]

/tmp/ipykernel_9006/811745436.py:9: UserWarning: semantic_masks_cleaned/0.tiff is a low contrast image
  imsave(f'semantic_masks_cleaned/{n}.tiff', semantic_mask)


  0%|          | 0/683 [00:00<?, ?it/s]

/tmp/ipykernel_9006/811745436.py:9: UserWarning: semantic_masks_cleaned/1.tiff is a low contrast image
  imsave(f'semantic_masks_cleaned/{n}.tiff', semantic_mask)


...

/tmp/ipykernel_9006/811745436.py:9: UserWarning: semantic_masks_cleaned/2.tiff is a low contrast image
  imsave(f'semantic_masks_cleaned/{n}.tiff', semantic_mask)


  0%|          | 0/773 [00:00<?, ?it/s]

/tmp/ipykernel_9006/811745436.py:9: UserWarning: semantic_masks_cleaned/3.tiff is a low contrast image
  imsave(f'semantic_masks_cleaned/{n}.tiff', semantic_mask)


  0%|          | 0/793 [00:00<?, ?it/s]


KeyboardInterrupt



In [29]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [8]:
### convert to dask array so that it is compatible with images for localisation
masks = da.from_array(masks)

In [9]:
masks

dask.array<array, shape=(75, 6048, 6048), dtype=uint16, chunksize=(75, 864, 864), chunktype=numpy.ndarray>

# Localise raw masks

Include size filter and measurements of fluorescence

In [11]:
feat = [
      "area",
      "major_axis_length",
      "minor_axis_length",
      "orientation",
      "mean_intensity",
        ]

In [25]:
objects = btrack.utils.segmentation_to_objects(
    masks, 
    images,
    properties = tuple(feat),
    use_weighted_centroid = False, 
)

[INFO][2023/01/19 04:56:11 PM] Localizing objects from segmentation...
[INFO][2023/01/19 04:56:11 PM] Found intensity_image data
[INFO][2023/01/19 05:12:21 PM] Objects are of type: <class 'dict'>
[INFO][2023/01/19 05:12:21 PM] ...Found 89235 objects in 75 frames.


In [26]:
objects = [o for o in objects if o.properties['area'] > 2500]

#### Find the efd?

In [ ]:
### finding the EFD
for obj in tqdm(objects):
    ### extract the intensity image (1ch only)
    glimpse = obj.properties['intensity_image'][...,0]
    ### pad the glimpse to ensure only one object is identifiable
    glimpse = np.pad(glimpse, pad_width = 1)
    ### find the contours (zero because only one object)
    contours = skimage.measure.find_contours(glimpse, fully_connected='high', level = 0.5)[0]
    ### get the efd
    efd = elliptic_fourier_descriptors(contours, order=100, normalize=True)
#     obj.properties = {'efd': efd}
    flatten_efd = efd.flatten()
    obj.properties = {'efd flat': flatten_efd}

In [27]:
with btrack.dataio.HDF5FileHandler(
     'objects.h5', 'w', obj_type='obj_type_1',
) as hdf:
    hdf.write_segmentation(masks)
    hdf.write_objects(objects)

[INFO][2023/01/19 05:12:22 PM] Opening HDF file: objects.h5...
[INFO][2023/01/19 05:13:04 PM] Writing objects/obj_type_1
[INFO][2023/01/19 05:13:04 PM] Writing labels/obj_type_1
[INFO][2023/01/19 05:13:04 PM] Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/area (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/major_axis_length (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/minor_axis_length (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/orientation (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/mean_intensity-0 (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/mean_intensity-1 (40381,)
[INFO][2023/01/19 05:13:09 PM] Closing HDF file: objects.h5


In [29]:
objects[0]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,minor_axis_length,orientation,mean_intensity-0,mean_intensity-1
0,0,257.838564,104.539205,0.0,0,False,7,5,36194,244.63143,204.217784,-0.625041,1751.473891,343.588302
